[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/tmollick95/iMapInvasives/blob/master/Code/Data%20download%20and%20preprocess%20code.ipynb)

## Leveraging machine learning to confirm invasive species reports

In [1]:
#attach to Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Install required Python libraries


In [2]:
!pip install geopandas
!pip install seaborn
!pip install earthpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 53.7 MB/s eta 0:00:00


### Importing python librariers

In [7]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import os
import requests
import matplotlib.pyplot as plt
%matplotlib inline
from PIL import Image
from io import BytesIO
import math
import json
from IPython.display import display, HTML

### Request iMapInvasives API with run in Cookie based system
#### Website: https://www.imapinvasives.org/

In [5]:
import requests

imap_site = "imapinvasives"
iMap_username = "tmollick@esf.edu"
iMap_password = "URPku@130436"

login_url = r"https://{0}.natureserve.org/imap/j_spring_security_check".format(imap_site)

# Attempt to log in
print("\nAttempting to authenticate with: {0}".format(login_url))
iMapSession = requests.Session()  # This is a global variable accessed in later functions
login_response = iMapSession.post(login_url,{'j_username':iMap_username,'j_password':iMap_password})
login_response_message = "\nlogin response: {0}".format(login_response.status_code)
print(login_response_message)
login_response.raise_for_status()

# Attempt to access a record to check if log-in was successful
test_aoi_url = r"https://{0}.natureserve.org/imap/services/aoi/new".format(imap_site)
test_aoi_record = iMapSession.get(test_aoi_url)
test_aoi_record_message = "\nTest record access response: {0}".format(test_aoi_record.status_code)
print(test_aoi_record_message)

if test_aoi_record.status_code == 403:
    print("\nResponse Code 403 is most likely the result of an incorrectly entered iMap username or password.  It may also be caused by logging in as a user with insufficient permissions.")

test_aoi_record.raise_for_status()


Attempting to authenticate with: https://imapinvasives.natureserve.org/imap/j_spring_security_check

login response: 200

Test record access response: 200


### Fliter 500 Spotted Lanternfly Records

In [ ]:
import pandas as pd

# Load the CSV file
ny = pd.read_csv("New_York_iMap_records_from_year_2000.csv", encoding="ISO-8859-1")

# Filter records where jurisdicti is "New York"
ny_records = ny[ny['jurisdicti'] == 'New York']

# Further filter the records for "Alliaria petiolata"
alliaria_records = ny_records[ny_records['scientific'] == 'Lycorma delicatula']

# Check the available counties for this species in New York
available_counties = alliaria_records['county'].unique()
print(f"Available counties for Lycorma delicatula in New York: {available_counties}")

# Evenly distribute 500 records across all counties
num_counties = len(available_counties)
records_per_county = 500 // num_counties

# Sample records evenly across counties
sampled_records = alliaria_records.groupby('county').apply(lambda x: x.sample(n=min(len(x), records_per_county), replace=False))

# If we still need more records, sample remaining from counties that have more data
remaining_records_needed = 500 - len(sampled_records)

if remaining_records_needed > 0:
    extra_records = alliaria_records[~alliaria_records.index.isin(sampled_records.index)].sample(n=remaining_records_needed, replace=False)
    sampled_records = pd.concat([sampled_records, extra_records])

# Remove multi-index caused by groupby
sampled_records = sampled_records.reset_index(drop=True)

# Save the output to CSV
output_file = "Lycorma delicatula_500_records_New_York.csv"
sampled_records.to_csv(output_file, index=False)

print(f"500 records of 'Lycorma delicatula' from New York saved to {output_file}")


### Filter iMap records which have photos uploaded by users

In [ ]:
# Read the presence IDs from the CSV file
record_ids = pd.read_csv("Lycorma delicatula_500_records_New_York.csv")
pre_ID = record_ids["presence_i"].tolist()  # Convert Series to list

In [6]:
record_ids.head()

In [ ]:
base_url = f"https://{imap_site}.natureserve.org/imap/services/presence/"

# List to store presence IDs with at least one species having a photo
presence_ids_with_photos = []

# Iterate over each presence ID
for presence_id in pre_ID:
    # Construct the full URL for each presence ID
    full_url = f"{base_url}{presence_id}"

    # Get the iMap record for this presence ID
    try:
        imap_record = iMapSession.get(full_url)  # Use the session to maintain authentication

        # Check if the request was successful
        if imap_record.status_code == 200:
            # Convert the response to a dictionary
            imap_dictionary = imap_record.json()

            # Extract all presentSpeciesId values from the speciesList
            present_species_ids = [species.get("presentSpeciesId") for species in imap_dictionary.get("speciesList", [])]

            # Filter out any None values just in case
            present_species_ids = [id for id in present_species_ids if id is not None]

            # Check if any presentSpeciesId has hasPhotoInd == True
            has_photo = any(species.get("hasPhotoInd") == True for species in imap_dictionary.get("speciesList", []))

            # If at least one presentSpeciesId has a photo, add the presence ID to the list
            if has_photo:
                presence_ids_with_photos.append(presence_id)

    except requests.exceptions.RequestException as e:
        print(f"An error occurred while processing presence ID {presence_id}: {e}")

In [ ]:
print(presence_ids_with_photos)

In [ ]:
presence_ids = presence_ids_with_photos

### Plot iMap record photos

In [ ]:
import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt
import math

# Function to get the image from a URL, handling redirects and setting headers
def get_image_from_url(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    # Perform the GET request with headers and follow redirects
    response = iMapSession.get(url, headers=headers, allow_redirects=True)

    if response.status_code == 200:
        try:
            return Image.open(BytesIO(response.content))
        except Exception as e:
            print(f"Failed to open image. Error: {e}")
            return None
    else:
        print(f"Failed to retrieve image. Status code: {response.status_code}")
        return None

# Function to plot images for each presentSpeciesId in a grid with 3 columns
def plot_images_for_species(imap_dictionary, presence_id):
    species_list = imap_dictionary.get("speciesList", [])
    num_species = len(species_list)
    num_columns = 3
    num_rows = math.ceil(num_species / num_columns)

    fig, axs = plt.subplots(num_rows, num_columns, figsize=(15, 5 * num_rows))
    axs = axs.flatten()  # Flatten the 2D array of axes to 1D for easy iteration

    for i, species in enumerate(species_list):
        img = None

        # Check if the 'photos' list exists and has at least one item
        if "photos" in species and species["photos"]:
            photo_url = species["photos"][0].get("photoUrl", "")

            if photo_url and (photo_url.lower().endswith('.jpg') or photo_url.lower().endswith('.jpeg')):  # Ensure the URL is a valid .jpg or .jpeg
                img = get_image_from_url(photo_url)

        # Display the image if it was successfully retrieved
        if img:
            axs[i].imshow(img)
            axs[i].axis('off')  # Turn off the axis
            axs[i].set_title(f"ID: {presence_id}\nSpecies ID: {species.get('presentSpeciesId')}")
        else:
            axs[i].text(0.5, 0.5, f'ID: {presence_id}\nImage Not Available', horizontalalignment='center', verticalalignment='center')
            axs[i].axis('off')

    # Turn off remaining empty subplots
    for j in range(i + 1, num_rows * num_columns):
        axs[j].axis('off')

    plt.tight_layout()
    plt.show()

# Function to process multiple presence IDs and plot images
def plot_images_for_multiple_species_ids(presence_ids):
    base_url = "https://imapinvasives.natureserve.org/imap/services/presence/"

    for presence_id in presence_ids:
        print(f"\nProcessing presence ID: {presence_id}")
        # Construct the full URL for each presence ID
        full_url = f"{base_url}{presence_id}"

        # Get the iMap record for this presence ID
        imap_record = iMapSession.get(full_url)

        if imap_record.status_code == 200:
            # Convert the response to a dictionary
            imap_dictionary = imap_record.json()

            # Plot images for this presence ID
            plot_images_for_species(imap_dictionary, presence_id)
        else:
            print(f"Failed to retrieve data for presence ID {presence_id}. Status code: {imap_record.status_code}")

# Process all the presence IDs and plot images
plot_images_for_multiple_species_ids(presence_ids)
